# Create Memory Resource

## Short term memory

In [ ]:
from bedrock_agentcore.memory import MemoryClient

client = MemoryClient(region_name="us-east-1")

memory = client.create_memory(
    name="PokemonAgentMemory",
    description="Memory for pokemon agent",
    event_expiry_days=30 # default to 90 days
)

# The memory_id will be used in following operations
print(f"Memory ID: {memory.get('id')}")
print(f"Memory: {memory}")
memory_id = memory.get('id')

## Long term memory

To [set up long term memory](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/memory-getting-started.html#memory-getting-started-long-term-memory), we can either 
- Add strategy to exisitng memory, or 
- pass in `strategies` parameter while creating a new memory with `create_memory_and_wait`. 

<br>
Couple notes: 

- Long-term memory takes 2-3 minutes to become ACTIVE.
- Long-term memory records will only be extracted from events that are stored after the newly added strategies become ACTIVE. And Conversations stored before a strategy is added will not appear in long-term memory.

In [ ]:
# there is also the add_semantic_strategy, add_user_preference_strategy, add_custom_semantic_strategy and their and_wait version.
# to update, add, and delete all together, we can also use the update_memory_strategies_and_wait function
client.add_summary_strategy_and_wait(
    memory_id=memory_id,
    name="SessionSummarizer",
    description="Summarizes conversation sessions",
    # Namespace allow us to organize all extracted information.
    # This template will extract information for each sessionId belonging to an actor in separate namespace
    #
    # *important*:
    # - the type of namespaces here is a list, but if we pass in multiple, we will get the following excpetion
    #   ValidationException: An error occurred (ValidationException) when calling the UpdateMemory operation: Validation failed during UpdateMemory: Memory strategy SessionSummarizer has more than one namespace
    # - Both actorId and sessionId are required for summary strategy.
    # That is we cannot leave out the session_id parameter, ie: /summaries/{actorId}, so that the summary can be per-user.
    namespaces=["/summaries/{actorId}/{sessionId}"],
)

{'arn': 'arn:aws:bedrock-agentcore:us-east-1:075198889659:memory/PokemonAgentMemory-zCM04KDWbq',
 'id': 'PokemonAgentMemory-zCM04KDWbq',
 'name': 'PokemonAgentMemory',
 'description': 'Memory for pokemon agent',
 'eventExpiryDuration': 90,
 'status': 'ACTIVE',
 'createdAt': datetime.datetime(2025, 7, 30, 8, 39, 14, 594000, tzinfo=tzlocal()),
 'updatedAt': datetime.datetime(2025, 7, 31, 9, 55, 9, 854000, tzinfo=tzlocal()),
 'strategies': [{'strategyId': 'SessionSummarizer-ObDqo57HqT',
   'name': 'SessionSummarizer',
   'description': 'Summarizes conversation sessions',
   'type': 'SUMMARIZATION',
   'namespaces': ['/summaries/{actorId}/{sessionId}'],
   'createdAt': datetime.datetime(2025, 7, 31, 9, 55, 9, 884000, tzinfo=tzlocal()),
   'updatedAt': datetime.datetime(2025, 7, 31, 9, 55, 9, 884000, tzinfo=tzlocal()),
   'status': 'ACTIVE',
   'memoryStrategyId': 'SessionSummarizer-ObDqo57HqT',
   'memoryStrategyType': 'SUMMARIZATION'}],
 'memoryId': 'PokemonAgentMemory-zCM04KDWbq',
 'memo